In [1]:
import os
import pandas as pd
import numpy as np
import pathlib as pth
import numbers
import sys

In [2]:
TRADE_PATH = pth.Path('input/trade/')
ALL_COUNTRIES = set(f.split('.')[0] for f in os.listdir(TRADE_PATH))
GDP_PATH = pth.Path('input/')

print(f"""{TRADE_PATH=}

{GDP_PATH=}

{ALL_COUNTRIES=}
""")

TRADE_PATH=PosixPath('input/trade')

GDP_PATH=PosixPath('input')

ALL_COUNTRIES={'Switzerland', 'UnitedStates', 'Norway', 'Singapore', 'Spain', 'Denmark', 'Belgium', 'France', 'Brazil', 'BelgiumLuxembourg', 'Sweden', 'Greece', 'Mexico', 'Austria', 'UnitedKingdom', 'Korea', 'China', 'Portugal', 'NewZealand', 'Canada', 'Finland', 'Hong Kong', 'Japan', 'Australia', 'Ireland', 'Germany', 'Italy', 'NetherlandsThe'}



In [3]:
def clean_trade(df):
    df = df.T
    df = df.drop(df.index[0])
    df = df.rename(columns=df.iloc[0])
    df = df.drop(df.index[0])
    df = df.rename(columns={np.nan: 'Year'})
    # Determine whether dates are yearly or quarterly
    print(df)
    first_year = df['Year'].iloc[0]
    if isinstance(first_year, numbers.Number):
        df = df.astype({'Year': int})
    elif isinstance(first_year, str) and 'Q' in first_year:
        df = df.astype({'Year': str})
    else:
        raise NotImplemented(f"Year has unexpected format {first_year} and type {type(first_year)}")
    df = df.set_index('Year')
    return df


def make_trade(path):
    dfs = {'import': {}, 'export': {}}
    for root, _, files in os.walk(path):
        for file in map(pth.Path, files):
            path = pth.Path(root)/file
            print(path, path.stem)
            
            for sheet in ('Exports, FOB', 'Imports, CIF'):
                df = pd.read_excel(path, skiprows=[1, 2, 3, 4, 5, 7], header=0, sheet_name=sheet)
                trade = sheet.split()[0].lower()[:-2]
                dfs[trade][path.stem] = clean_trade(df)
                
    return dfs

In [4]:
# trade_dfs = make_trade(TRADE_PATH)

In [6]:
def clean_gdp(df):
    df = df.drop(columns='Unnamed: 0')
    df = df.T
    df = df.rename(columns=df.iloc[0])
    df = df.drop(df.index[0])
    df = df.rename(columns={'Country': 'Year'})
    # Determine whether dates are yearly or quarterly
    first_year = df['Year'].iloc[0]
    if isinstance(first_year, numbers.Number):
        df = df.astype({'Year': int})
    elif isinstance(first_year, str) and 'Q' in first_year:
        df = df.astype({'Year': str})
    else:
        raise NotImplemented(f"Year has unexpected format {first_year} and type {type(first_year)}")
    df = df.set_index('Year')
    print(df)
    df = df.filter(items=ALL_COUNTRIES)
    assert len(df.columns) == len(ALL_COUNTRIES), f'{ALL_COUNTRIES.difference(df.columns)=}'
    print(df)
    return df

def make_gdp(path):
    dfs = {}
    for root, _, files in os.walk(path):
        for file in filter(lambda p: 'RealGDP' in p.stem, map(lambda f: pth.Path(root)/f, files)):
            print(file)
            df = pd.read_excel(file, skiprows=[1, 2, 3, 4, 5], header=0)
            df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3'])
            df = clean_gdp(df)
            # trade = sheet.split()[0].lower()[:-2]
            # dfs[path.stem] = clean_trade(df)

In [7]:
gdp_dfs = make_gdp(GDP_PATH)

input/RealGDP.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
       Argentina Australia       Austria        Belgium    Brazil  \
Year                                                                
1959Q1       ...       ...           ...            ...       ...   
1959Q2       ...       ...           ...            ...       ...   
1959Q3       ...     62496           ...            ...       ...   
1959Q4       ...     63043           ...            ...       ...   
1960Q1       ...     67915           ...            ...       ...   
...          ...       ...           ...            ...       ...   
2020Q4  163301.7    501813   79734.73078  100043.966054  299623.4   
2021Q1  169125.2    511583  78915.891427  101361.263667  302822.6   
2021Q2  168080.6    514593  82214.082169  103095.968922    301977   
2021Q3  174578.4    505177  85658.134642  105262.956032  302156.5   
2021Q4  177426.6    524644  84562.481251  105711.971379  304497.9   

           

AssertionError: ALL_COUNTRIES.difference(df.columns)={'NewZealand', 'UnitedStates', 'BelgiumLuxembourg', 'Hong Kong', 'UnitedKingdom', 'Korea', 'China', 'NetherlandsThe'}

In [113]:
def result(dataframes):
    """ Each row has the following sequence:
    Country, Year (four times, one per quarter), ...
    """

    assert sys.version_info.major >= 3 and sys.version_info.minor >= 9, "Solution depends on dicts being ordered"

    result = []
    for country in ALL_COUNTRIES:
        lines = {}
        needs_padding = False
        for trade, dfs in dataframes.items():
            df = dfs[country]
            # Check if data was simple int year of quarterly "1960Q1"
            first_year = df.index[0]
            needs_padding = isinstance(first_year, numbers.Number)
            if needs_padding:
                year_gen = lambda y: f'{y}1'
            else:
                assert isinstance(first_year, str) and 'Q' in first_year, f'Unexpected {first_year=}, {type(first_year)}'  
                year_gen = lambda y: y.replace('Q', '')
               
            for year, trade_to_country in df.iterrows():
                lines.setdefault(year, {}).update(
                    {'Year': year_gen(year)} | {f'Export to {k}' if trade == 'export' else f'Import from {k}': v for k, v in trade_to_country.items()}
                )
        for year, data in lines.items():
            result.append({'Country': country, **data})
            if needs_padding:
                for i in range(2, 5):
                    result.append({'Country': country, 'Year': f'{year}{i}'})
    print(result)
    return pd.DataFrame(result)

In [114]:
df = result(trade_dfs)
pd.set_option('display.max_rows', 6000)
#df[df['Country'] == 'Brazil']
df

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



,Country,Year,Import from World,Import from Advanced Economies,Import from Euro Area,Import from Austria,Import from Belgium,Import from Belgium-Luxembourg,Import from Cyprus,"Import from Estonia, Rep. of",...,Export to Emerging Market and Developing Economies by Source of Export Earnings: Fuel,Export to Emerging Market and Developing Economies by Source of Export Earnings: Nonfuel,"Import from Kosovo, Rep. of",Import from Turkey,Import from Yemen Arab Rep.,Import from Middle East and Central Asia not specified,Import from Brazil,"Export to Kosovo, Rep. of",Export to Turkey,Export to Brazil
0,Brazil,19601,345.070000,238.700000,84.920000,0.77,NaN,4.84,NaN,NaN,...,1.000000,40.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brazil,19602,380.930000,266.090000,106.260000,0.66,NaN,4.07,NaN,NaN,...,1.200000,43.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brazil,19603,436.810000,306.130000,94.050000,0.66,NaN,4.29,NaN,NaN,...,9.100000,44.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Brazil,19604,423.940000,291.060000,102.190000,1.1,NaN,4.51,NaN,NaN,...,5.700000,44.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Brazil,19611,376.310000,272.360000,84.810000,0.88,NaN,3.96,NaN,NaN,...,0.600000,43.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6496,Singapore,20213,119675.650970,50008.510560,9308.312048,222.35 r,329.68 r,NaN,4.50 r,42.01 r,...,2069.777540,56829.892404,NaN,NaN,NaN,NaN,"1,117.54 r",NaN,NaN,420.32 r
6497,Singapore,20214,136870.763764,54507.146395,10182.493499,247.50 r,341.58 r,NaN,4.94 r,55.26 r,...,2303.666993,62884.242394,NaN,NaN,NaN,NaN,"1,171.15 r",NaN,NaN,471.70 r
6498,Singapore,20221,130753.375943,52899.320152,9870.823899,239.17 r,335.18 r,NaN,4.79 r,51.72 r,...,2384.445530,65266.044999,NaN,NaN,NaN,NaN,"1,144.46 r",NaN,NaN,488.17 r
6499,Singapore,20222,130593.525981,54106.555008,10078.743354,242.77 r,349.33 r,NaN,4.89 r,49.48 r,...,2523.854568,69334.899163,NaN,NaN,NaN,NaN,"1,185.85 r",NaN,NaN,516.09 r


In [87]:
df.to_excel('result.xlsx')
print('Done')

Done
